## Imports

In [114]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [115]:
import os, sys
from pathlib import Path
sys.path.append('/home/k64835/Master-Thesis-SITS')

scripts_path = Path("../Data-Preprocessing/").resolve()
sys.path.append(str(scripts_path))

scripts_path = Path("../Evaluation/").resolve()
sys.path.append(str(scripts_path))

In [116]:
import pickle
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestCentroid
from scripts.data_visualiser import *
from scripts.data_loader import *
from scripts.data_preprocessor import *
from scripts.temporal_data_preprocessor import *
from scripts.temporal_data_loader import *
from scripts.temporal_visualiser import *
from scripts.temporal_chanel_refinement import *
from model_scripts.model_helper import *
from model_scripts.dataset_creation import *
from model_scripts.train_model_ae import *
from model_scripts.model_visualiser import *
from model_scripts.subpatch_extraction import *
from model_scripts.clustering import *
from evaluation_scripts.result_visualiser import *
from evaluation_scripts.evaluation_helper import *
from model_scripts.pretrained_temporal_feature_extraction import *
from Pipeline.temporal_preprocessing_pipeline import *
from Pipeline.temporal_preprocessing_pipeline import *
import numpy as np
import config as config
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
import skimage.measure
import torch
import torch.nn as nn
import torch.optim as optim

## Prepare dataset: B10

### Loading the pre-processed data

Data: Extracted and Pre-processed Patches (each patch containing a sugarbeet field)

Dimensions: (N, T, C, H, W) = (N, 7, 10, 64, 64)

In [75]:
preprocessing_pipeline = PreProcessingPipelineTemporal()
field_numbers_train, acquisition_dates_train, patch_tensor_train, visualisation_train = preprocessing_pipeline.get_processed_temporal_cubes('train', 'b10')
field_numbers_eval, acquisition_dates_eval, patch_tensor_eval, visualisation_eval = preprocessing_pipeline.get_processed_temporal_cubes('eval', 'b10')
patch_tensor_train.shape, patch_tensor_eval.shape

(torch.Size([2425, 7, 10, 64, 64]), torch.Size([48, 7, 10, 64, 64]))

### Create Sub-Patches

In [76]:
train_subpatches, train_subpatch_coords = non_overlapping_sliding_window(patch_tensor_train, field_numbers_train, patch_size=config.subpatch_size)
eval_subpatches, eval_subpatch_coords = non_overlapping_sliding_window(patch_tensor_eval, field_numbers_eval, patch_size=config.subpatch_size)
train_subpatches.shape, eval_subpatches.shape

(torch.Size([54623, 7, 10, 4, 4]), torch.Size([1197, 7, 10, 4, 4]))

Get properly formatted field numbers for the evaluation function

In [83]:
train_coord_fn = get_string_fielddata(train_subpatch_coords)
eval_coord_fn = get_string_fielddata(eval_subpatch_coords)

## Experiment 1: Feature Extraction using Channel-wise Histrograms

### Feature extraction 

In [54]:
histogram_features_train = extract_channel_histograms(train_subpatches, bins=256)
histogram_features_eval = extract_channel_histograms(eval_subpatches, bins=256)
histogram_features_train.shape # (N, T, C * bins)

(54623, 17920)

### Modeling: k-means

Clustering the sub-patches 

In [55]:
kmeans_b10 = kmeans_function(histogram_features_train, n_clusters=2, random_state=31)

train_subpatch_predictions = kmeans_b10.predict(histogram_features_train.reshape(histogram_features_train.shape[0],-1))
eval_subpatch_predictions = kmeans_b10.predict(histogram_features_eval.reshape(histogram_features_eval.shape[0],-1))

### Evaluation

Ensure all predictions are not zeros/ones

In [56]:
np.unique(eval_subpatch_predictions)

array([0, 1], dtype=int32)

Clustering Accuracy

In [57]:
disease, acc, precision, recall, f1_score, f2_score = evaluate_clustering_metrics(eval_coord_fn, eval_subpatch_predictions, config.labels_path, config.subpatch_to_patch_threshold)
print("Disease cluster:", disease)
print("Accuracy:",acc)
print("Precision:",precision)
print("Recall:",recall)
print("F1-score:",f1_score)
print("F2-score:", f2_score)

Disease cluster: 0
Accuracy: 47.54
Precision: 53.85
Recall: 60.0
F1-score: 56.76
F2-score: 58.66


Save Model

In [59]:
# with open(config.kmeans_hist_path, 'wb') as file:
#     pickle.dump(kmeans_b10, file)

## Experiment 2: Feature Reduction using PCA

### Feature extraction 

In [118]:
pca, features_train, top_channel_indices_train = pca_feature_extraction(train_subpatches) 
pca, features_eval, top_channel_indices_eval = pca_feature_extraction(eval_subpatches)

In [127]:
# top_channel_indices_train

### Modeling: k-means

Clustering the sub-patches 

In [121]:
kmeans_b10 = kmeans_function(np.array(features_train), n_clusters=2, random_state=32)

train_subpatch_predictions = kmeans_b10.predict(np.array(features_train))
eval_subpatch_predictions = kmeans_b10.predict(np.array(features_eval))

### Evaluation

Ensure all predictions are not zeros/ones

In [122]:
np.unique(eval_subpatch_predictions)

array([0, 1], dtype=int32)

Clustering Accuracy

In [123]:
disease, acc, precision, recall, f1_score, f2_score = evaluate_clustering_metrics(eval_coord_fn, eval_subpatch_predictions, config.labels_path, config.subpatch_to_patch_threshold)
print("Disease cluster:", disease)
print("Accuracy:",acc)
print("Precision:",precision)
print("Recall:",recall)
print("F1-score:",f1_score)
print("F2-score:", f2_score)

Disease cluster: 1
Accuracy: 57.38
Precision: 64.52
Recall: 57.14
F1-score: 60.61
F2-score: 58.48


Save Model

In [129]:
# with open(config.kmeans_pca_path, 'wb') as file:
#     pickle.dump(kmeans_b10, file)

## Experiment 3: Feature Extraction using Vegetation Indices

### Data Loading + Feature Extraction
#### Loading the pre-processed data with Vegetation Indices = NDVI, MSI, EVI

Dimensions: (N, T, C, H, W) = (N, 7, 3, 64, 64)

In [ ]:
preprocessing_pipeline = PreProcessingPipelineTemporal()
field_numbers_train, acquisition_dates_train, patch_tensor_train, visualisation_train = preprocessing_pipeline.get_processed_temporal_cubes('train', 'mvi')
field_numbers_eval, acquisition_dates_eval, patch_tensor_eval, visualisation_eval = preprocessing_pipeline.get_processed_temporal_cubes('eval', 'mvi')
patch_tensor_train.shape, patch_tensor_eval.shape

(torch.Size([2425, 7, 3, 64, 64]), torch.Size([48, 7, 3, 64, 64]))

### Create Sub-Patches

In [ ]:
train_subpatches, train_subpatch_coords = non_overlapping_sliding_window(patch_tensor_train, field_numbers_train, patch_size=config.subpatch_size)
eval_subpatches, eval_subpatch_coords = non_overlapping_sliding_window(patch_tensor_eval, field_numbers_eval, patch_size=config.subpatch_size)
train_subpatches.shape, eval_subpatches.shape

(torch.Size([54623, 7, 3, 4, 4]), torch.Size([1197, 7, 3, 4, 4]))

Get properly formatted field numbers for the evaluation function

In [ ]:
train_coord_fn = get_string_fielddata(train_subpatch_coords)
eval_coord_fn = get_string_fielddata(eval_subpatch_coords)

### Modeling: k-means

Clustering the sub-patches 

In [ ]:
kmeans_b10 = kmeans_function(train_subpatches, n_clusters=2, random_state=32)

train_subpatch_predictions = kmeans_b10.predict(train_subpatches.reshape(train_subpatches.size(0), -1).numpy())
test_subpatch_predictions = kmeans_b10.predict(eval_subpatches.reshape(eval_subpatches.size(0), -1).numpy())

### Evaluation

Ensure all predictions are not zeros/ones

In [ ]:
np.unique(eval_subpatch_predictions)

array([0, 1], dtype=int32)

Clustering Accuracy

In [ ]:
disease, acc, precision, recall, f1_score, f2_score = evaluate_clustering_metrics(eval_coord_fn, test_subpatch_predictions, config.labels_path, config.subpatch_to_patch_threshold)
print("Disease cluster:", disease)
print("Accuracy:",acc)
print("Precision:",precision)
print("Recall:",recall)
print("F1-score:",f1_score)
print("F2-score:", f2_score)

Disease cluster: 0
Accuracy: 47.54
Precision: 53.49
Recall: 65.71
F1-score: 58.97
F2-score: 62.84


Save Model

In [ ]:
# with open(config.kmeans_vi_path, 'wb') as file:
#     pickle.dump(kmeans_b10, file)